<a href="https://colab.research.google.com/github/VamsiKrishnaMannava/myspark/blob/main/Pyspark_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=3313c3a3378022d35e67b54423d3e60d7daa6b1727afdd0e63c7276a78b6132f
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col

In [5]:
data = ["1,Alice\t30|New York","2,Jonny\t36|New Addington",]

In [10]:
spark = SparkSession.builder.appName(name="Test-1").getOrCreate()

In [14]:
df = spark.createDataFrame(data, "string")
df.show()

+--------------------+
|               value|
+--------------------+
|1,Alice\t30|New York|
|2,Jonny\t36|New A...|
+--------------------+



In [19]:
split_col = split(df['value'],',|\t|\|')

Column<'split(value, ,|	|\|, -1)'>


In [20]:
df = df.withColumn('id',split_col.getItem(0)).withColumn('name',split_col.getItem(1)).withColumn('age',split_col.getItem(2)).withColumn('city', split_col.getItem(3))

In [24]:
df.columns

['value', 'id', 'name', 'age', 'city']

In [28]:
df.show()

+--------------------+---+-----+---+-------------+
|               value| id| name|age|         city|
+--------------------+---+-----+---+-------------+
|1,Alice\t30|New York|  1|Alice| 30|     New York|
|2,Jonny\t36|New A...|  2|Jonny| 36|New Addington|
+--------------------+---+-----+---+-------------+



In [27]:
df.select('id', 'name', 'age', 'city').show()

+---+-----+---+-------------+
| id| name|age|         city|
+---+-----+---+-------------+
|  1|Alice| 30|     New York|
|  2|Jonny| 36|New Addington|
+---+-----+---+-------------+

